**Distiller Installation on Drive**

In [15]:
from google.colab import drive
ROOT="/content/drive"
print(ROOT)
drive.mount(ROOT)

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Clone the git repo on your drive
!git clone https://github.com/NervanaSystems/distiller.git

I had created a folder named 'distiller' already. Realized later that it was of no need. 
So you would probably require this: %cd drive/My Drive/GE Edge Capstone 2020/distiller.  

In [2]:
%cd drive/My Drive/GE Edge Capstone 2020/distiller/distiller

/content/drive/My Drive/GE Edge Capstone 2020/distiller/distiller


In [4]:
!pip3 install -e .

Obtaining file:///content/drive/My%20Drive/GE%20Edge%20Capstone%202020/distiller/distiller
     |████████████████████████████████| 2.1MB 4.6MB/s 
     |████████████████████████████████| 734.6MB 25kB/s 
     |████████████████████████████████| 10.2MB 47.4MB/s 
     |████████████████████████████████| 460kB 49.7MB/s 
     |████████████████████████████████| 110.5MB 114kB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 798kB 27.4MB/s 
     |████████████████████████████████| 112kB 52.1MB/s 
     |████████████████████████████████| 1.7MB 44.2MB/s 
     |████████████████████████████████| 235kB 54.3MB/s 
     |████████████████████████████████| 153kB 56.4MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 6.7MB 41.0MB/s 
     |████████████████████████████████| 1.5MB 40.9MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████

In [1]:
import distiller

In [2]:
%pwd

'/content'

In [3]:
%cd drive/My Drive/GE Edge Capstone 2020/distiller/distiller/examples/classifier_compression

/content/drive/My Drive/GE Edge Capstone 2020/distiller/distiller/examples/classifier_compression


**Loading the CIFAR-10 Dataset**

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset_CIFER = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

Files already downloaded and verified


'\ntrainloader_CIFER = torch.utils.data.DataLoader(trainset_CIFER, batch_size=4,\n                                          shuffle=True, num_workers=2)\n'

In [9]:
#Cites all the Arguments which could be passed
!python3 compress_classifier.py --help

usage: compress_classifier.py [-h] [--arch ARCH] [-j N] [--epochs N] [-b N]
                              [--lr LR] [--momentum M] [--weight-decay W]
                              [--print-freq N] [--verbose]
                              [--resume-from PATH | --exp-load-weights-from PATH]
                              [--pretrained] [--reset-optimizer] [-e]
                              [--activation-stats PHASE [PHASE ...]]
                              [--activation-histograms PORTION_OF_TEST_SET]
                              [--masks-sparsity] [--param-hist]
                              [--summary {sparsity,compute,model,modules,png,png_w_params}]
                              [--export-onnx [EXPORT_ONNX]]
                              [--compress [COMPRESS]]
                              [--sense {element,filter,channel}]
                              [--sense-range SENSITIVITY_RANGE SENSITIVITY_RANGE SENSITIVITY_RANGE]
                              [--deterministic] [--seed SEE

**Post Quantization Training** 

A list of architecture is defined for a specific dataset under the 'models' folder. Currently Distiller only supports Imagenet, CIFAR-10 and MNIST datasets. Imagenet however is not publicly available so loading the dataset is tedious. In order to experiment with architectures and datasets beyond distiller we would have to modify the codes, starting with 'models' and 'dataset loaders'. 'Modules' for distiller have also been defined to enable us to create our own architectures as per distiller (slight variation from pytorch)

In [ ]:
#Removed the pretrained argument which was given in the original command as received an error that the particular pretrained model is not available
!python3 compress_classifier.py -a resnet20_cifar './data'  --quantize-eval --evaluate

Log file for this run: /content/drive/My Drive/GE Edge Capstone 2020/distiller/distiller/examples/classifier_compression/logs/2020.10.12-154332/2020.10.12-154332.log
Random seed: 0

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> created a resnet20_cifar model with the cifar10 dataset
Files already downloaded and verified
Dataset sizes:
	test=10000
Files already downloaded and verified
Dataset sizes:
	test=500
Pass 1: Collecting min, max, avg_min, avg_max, mean
--- test ---------------------
500 samples (128 per mini-batch)
==> Top1: 9.600    Top5: 49.200    Loss: 3.473

Pass 2: Collecting b, std parameters
--- test ---------------------
500 samples (128 per mini-batch)
==> Top1: 9.600    Top5: 49.200    Loss: 3.479

Stats collection complete
Stats saved to logs/2020.10.12-154332/acts_quantization_stats.yaml
Preparing model for quantization using PostTrainLinearQuantizer
Applying batch

In [8]:
#Removed the pretrained argument
!python3 compress_classifier.py -a plain20_cifar './data'  --quantize-eval --evaluate

Log file for this run: /content/drive/My Drive/GE Edge Capstone 2020/distiller/distiller/examples/classifier_compression/logs/2020.10.13-192208/2020.10.13-192208.log
Random seed: 0

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> created a plain20_cifar model with the cifar10 dataset
Files already downloaded and verified
Dataset sizes:
	test=10000
Files already downloaded and verified
Dataset sizes:
	test=500
Pass 1: Collecting min, max, avg_min, avg_max, mean
--- test ---------------------
500 samples (128 per mini-batch)
==> Top1: 12.800    Top5: 52.000    Loss: 2.303

Pass 2: Collecting b, std parameters
--- test ---------------------
500 samples (128 per mini-batch)
==> Top1: 12.800    Top5: 52.000    Loss: 2.303

Stats collection complete
Stats saved to logs/2020.10.13-192208/acts_quantization_stats.yaml
Preparing model for quantization using PostTrainLinearQuantizer
Applying batc

**Quantization Aware Training**

'compress_classifier.py' is the common file used across all image classification problems. Different quantization techniques are applied through the compression scheduler which is saved as .yaml files. Besides varying the arguments from the command line, we can also make edits to the compression scheduler i.e. '.yaml' files, such as 'bits_activation', 'weights_activation', 'signed or unsigned weights' etc. This varies across schedulers. 

Path to access different compression schedulers built for QAT:
Capstone Project/distiller/examples/quantization/quant_aware_train



In [13]:
# 1. Quant_aware_train_linear_quant.yaml 
# Read more at : "https://nervanasystems.github.io/distiller/algo_quantization.html"
#Removed the argument 'pretrained'
!python compress_classifier.py -a resnet20_cifar -p 50 -b 256 './data' --epochs 10 --compress=../quantization/quant_aware_train/quant_aware_train_linear_quant.yaml -j 22 --lr 0.0001 --vs 0 --gpu 0

Log file for this run: /content/drive/My Drive/GE Edge Capstone 2020/distiller/distiller/examples/classifier_compression/logs/2020.10.13-193631/2020.10.13-193631.log
Random seed: 4863

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> created a resnet20_cifar model with the cifar10 dataset
Reading compression schedule from: ../quantization/quant_aware_train/quant_aware_train_linear_quant.yaml
Preparing model for quantization using QuantAwareTrainRangeLinearQuantizer
Files already downloaded and verified
Files already downloaded and verified
Dataset sizes:
	training=50000
	validation=10000
	test=10000


Training epoch: 50000 samples (256 per mini-batch)
Epoch: [0][   50/  196]    Overall Loss 2.437931    Objective Loss 2.437931    Top1 10.632812    Top5 52.015625    LR 0.000100    Time 0.209053    
Epoch: [0][  100/  196]    Overall Loss 2.395941    Objective Loss 2.395941    Top1 11.7460

In [14]:
# 2. preact_resnet_cifar_dorefa.yaml
#Pre-activation ResNet20 on CIFAR10 (base FP32 + DoReFa)
#Changed epochs to 10 from 200
!python compress_classifier.py -a preact_resnet20_cifar --lr 0.1 -p 50 -b 128 './data' -j 1 --epochs 10 --compress=../quantization/quant_aware_train/preact_resnet_cifar_dorefa.yaml --wd=0.0002 --vs=0 --gpus 0

Log file for this run: /content/drive/My Drive/GE Edge Capstone 2020/distiller/distiller/examples/classifier_compression/logs/2020.10.13-194437/2020.10.13-194437.log
Random seed: 91584

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> created a preact_resnet20_cifar model with the cifar10 dataset
Reading compression schedule from: ../quantization/quant_aware_train/preact_resnet_cifar_dorefa.yaml
Preparing model for quantization using DorefaQuantizer
Files already downloaded and verified
Files already downloaded and verified
Dataset sizes:
	training=50000
	validation=10000
	test=10000


Training epoch: 50000 samples (128 per mini-batch)
Epoch: [0][   50/  391]    Overall Loss 2.131540    Objective Loss 2.131540    Top1 19.171875    Top5 69.375000    LR 0.100000    Time 0.069262    
Epoch: [0][  100/  391]    Overall Loss 2.010012    Objective Loss 2.010012    Top1 24.015625    Top5 76.05